### 测试CUTE和Corrfunc差异

In [1]:
import numpy as np 
import joblib 

In [2]:
root_path = "/Raid6/1/xiaoliang/simulation/BIGMDPL/data/"

In [3]:
tpcf_corr = joblib.load(root_path + "tpCF-11_nbar1e-3.shiftz-nbar1e-3.x8ran0-150s0to150_120mu.pkl")

In [8]:
tpcf_corr["muedges"]

array([0.        , 0.00833333, 0.01666667, 0.025     , 0.03333333,
       0.04166667, 0.05      , 0.05833333, 0.06666667, 0.075     ,
       0.08333333, 0.09166667, 0.1       , 0.10833333, 0.11666667,
       0.125     , 0.13333333, 0.14166667, 0.15      , 0.15833333,
       0.16666667, 0.175     , 0.18333333, 0.19166667, 0.2       ,
       0.20833333, 0.21666667, 0.225     , 0.23333333, 0.24166667,
       0.25      , 0.25833333, 0.26666667, 0.275     , 0.28333333,
       0.29166667, 0.3       , 0.30833333, 0.31666667, 0.325     ,
       0.33333333, 0.34166667, 0.35      , 0.35833333, 0.36666667,
       0.375     , 0.38333333, 0.39166667, 0.4       , 0.40833333,
       0.41666667, 0.425     , 0.43333333, 0.44166667, 0.45      ,
       0.45833333, 0.46666667, 0.475     , 0.48333333, 0.49166667,
       0.5       , 0.50833333, 0.51666667, 0.525     , 0.53333333,
       0.54166667, 0.55      , 0.55833333, 0.56666667, 0.575     ,
       0.58333333, 0.59166667, 0.6       , 0.60833333, 0.61666

### 测试宇宙学转换函数核心速度

In [1]:
import numpy as np 
import math
from LSS_python.base import DA_jit, Hz_jit
from numba import njit 

In [2]:
@njit
def smu_cosmo_convert(s_f, mu_f, DA_f, DA_t, H_f, H_t):
    """s1: angular direction; s2: LOS direction
    f: fiducial; t: task
    """
    s2 = s_f * mu_f
    s1 = s_f * np.sqrt(1 - mu_f**2)
    alpha1 = DA_t / DA_f
    alpha2 = H_f / H_t
    s_t = np.sqrt((alpha1 * s1) ** 2 + (alpha2 * s2) ** 2)
    mu_t = alpha2 * s2 / (s_t + 1e-15)
    return s_t, mu_t

@njit
def smu_cosmo_convert_math(s_f, mu_f, DA_f, DA_t, H_f, H_t):
    """s1: angular direction; s2: LOS direction
    f: fiducial; t: task
    """
    s2 = s_f * mu_f
    s1 = s_f * math.sqrt(1 - mu_f**2)
    alpha1 = DA_t / DA_f
    alpha2 = H_f / H_t
    s_t = math.sqrt((alpha1 * s1) ** 2 + (alpha2 * s2) ** 2)
    mu_t = alpha2 * s2 / (s_t + 1e-15)
    return s_t, mu_t

In [3]:
DA_f = DA_jit(0.6, 0.3071, -1.0)
DA_t = DA_jit(0.6, 0.4071, -0.8)

Hz_f = Hz_jit(0.6, 0.3071, -1.0)
Hz_t = Hz_jit(0.6, 0.4071, -0.8)

In [4]:
s_f, mu_f = 10.0, 0.4
s_t1, mu_t1 = smu_cosmo_convert(s_f, mu_f, DA_f, DA_t, Hz_f, Hz_t)
s_t2, mu_t2 = smu_cosmo_convert_math(s_f, mu_f, DA_f, DA_t, Hz_f, Hz_t)

In [9]:
number = 50000 * 100

s_f_array = np.ones(number) * s_f
mu_f_array = np.ones(number) * mu_f
DA_f_array = np.ones(number) * DA_f
Hz_f_array = np.ones(number) * Hz_f
Hz_t_array = np.ones(number) * Hz_t
DA_t_array = np.ones(number) * DA_t

s_t_array, mu_t_array = smu_cosmo_convert(s_f_array, mu_f_array, DA_f_array, DA_t_array, Hz_f_array, Hz_t_array)

In [10]:
%%timeit -n 3 -r 5
s_t_array, mu_t_array = smu_cosmo_convert(s_f_array, mu_f_array, DA_f_array, DA_t_array, Hz_f_array, Hz_t_array)

181 ms ± 4.96 ms per loop (mean ± std. dev. of 5 runs, 3 loops each)


In [21]:
%%timeit -n 100 -r 5
s_t1, mu_t1 = smu_cosmo_convert(s_f, mu_f, DA_f, DA_t, Hz_f, Hz_t)

1.67 μs ± 102 ns per loop (mean ± std. dev. of 5 runs, 100 loops each)


In [22]:
%%timeit -n 100 -r 5
s_t2, mu_t2 = smu_cosmo_convert_math(s_f, mu_f, DA_f, DA_t, Hz_f, Hz_t)

1.62 μs ± 89 ns per loop (mean ± std. dev. of 5 runs, 100 loops each)


### 测试：积分函数

In [1]:
import numpy as np 
from LSS_python.base import traz 

In [2]:
x_array = np.linspace(0.0, 1*np.pi, 100)
V_array = np.sin(x_array)
print(traz(V_array, x_array))

1.9998321638939927


In [8]:
x_array = np.linspace(0, 2, 10)
y_array = np.linspace(0, 1, 10)
X_mesh, Y_mesh = np.meshgrid(x_array, y_array, indexing="ij")
V_array = X_mesh + Y_mesh
print(traz(V_array, x_array, y_array))

3.0


### 测试：二维结果获取函数测试 

In [ ]:
import numpy as np 
from LSS_python.tpcf import xismu 
from xitools import xismu as xismu_old 